# 📈 Notebook 4: MLflow Experiment Tracking

**Author:** Amey Talkatkar | **Course:** MLOps with Agentic AI

## 🎯 Learning Objectives
- Connect to MLflow tracking server (DagsHub)
- Log parameters, metrics, and artifacts
- Compare multiple experiment runs
- Use MLflow Model Registry
- Load models from registry
- Transition models through stages

## 🔥 The Problem (Solved!)
From Notebook 3, we had:
- 3 models trained
- No systematic tracking
- Can't reproduce experiments
- Can't compare hyperparameters easily

**MLflow solves all of this!**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported")
print(f"MLflow version: {mlflow.__version__}")

## Step 1: Configure MLflow

Set up connection to MLflow tracking server (DagsHub).

In [ ]:
# Option 1: DagsHub (Recommended)
# Get these from your .env file
MLFLOW_TRACKING_URI = os.getenv('MLFLOW_TRACKING_URI', 'https://dagshub.com/YOUR_USERNAME/sales-forecasting-mlops.mlflow')
MLFLOW_TRACKING_USERNAME = os.getenv('MLFLOW_TRACKING_USERNAME', 'YOUR_USERNAME')
MLFLOW_TRACKING_PASSWORD = os.getenv('MLFLOW_TRACKING_PASSWORD', 'YOUR_TOKEN')

# Set credentials
os.environ['MLFLOW_TRACKING_USERNAME'] = MLFLOW_TRACKING_USERNAME
os.environ['MLFLOW_TRACKING_PASSWORD'] = MLFLOW_TRACKING_PASSWORD

# Set tracking URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print(f"✅ MLflow configured")
print(f"   Tracking URI: {MLFLOW_TRACKING_URI}")

# Test connection
try:
    experiments = mlflow.search_experiments()
    print(f"   ✅ Connection successful! Found {len(experiments)} experiments")
except Exception as e:
    print(f"   ❌ Connection failed: {e}")
    print("   💡 Check your credentials in .env file")

## Step 2: Create Experiment

In [ ]:
# Set experiment (creates if doesn't exist)
EXPERIMENT_NAME = "sales_forecasting_notebook"
mlflow.set_experiment(EXPERIMENT_NAME)

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print(f"✅ Experiment: {EXPERIMENT_NAME}")
print(f"   ID: {experiment.experiment_id}")
print(f"   Artifact Location: {experiment.artifact_location}")

## Step 3: Load Data

In [ ]:
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').squeeze()
y_test = pd.read_csv('../data/processed/y_test.csv').squeeze()

print(f"✅ Data loaded: {len(X_train):,} train, {len(X_test):,} test")

## Step 4: Train with MLflow Tracking

### Model 1: Linear Regression

In [ ]:
with mlflow.start_run(run_name="linear_regression_v1") as run:
    print(f"🔹 Run ID: {run.info.run_id}")
    
    # Log parameters
    mlflow.log_param("model_type", "LinearRegression")
    mlflow.log_param("fit_intercept", True)
    
    # Train model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2_score", r2)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")
    
    # Log dataset info
    mlflow.log_param("train_samples", len(X_train))
    mlflow.log_param("test_samples", len(X_test))
    mlflow.log_param("n_features", X_train.shape[1])
    
    print(f"✅ Logged: RMSE=${rmse:,.2f}, R²={r2:.4f}")

### Model 2: Random Forest

In [ ]:
with mlflow.start_run(run_name="random_forest_v1") as run:
    print(f"🌲 Run ID: {run.info.run_id}")
    
    # Model parameters
    params = {
        "n_estimators": 100,
        "max_depth": 10,
        "min_samples_split": 20,
        "min_samples_leaf": 10,
        "random_state": 42
    }
    
    # Log all parameters
    mlflow.log_param("model_type", "RandomForest")
    mlflow.log_params(params)
    
    # Train
    model = RandomForestRegressor(**params, n_jobs=-1)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metrics({
        "rmse": rmse,
        "mae": mae,
        "r2_score": r2
    })
    
    # Log model
    mlflow.sklearn.log_model(model, "model")
    
    print(f"✅ Logged: RMSE=${rmse:,.2f}, R²={r2:.4f}")

### Model 3: XGBoost

In [ ]:
with mlflow.start_run(run_name="xgboost_v1") as run:
    print(f"⚡ Run ID: {run.info.run_id}")
    
    # Model parameters
    params = {
        "n_estimators": 100,
        "learning_rate": 0.1,
        "max_depth": 5,
        "min_child_weight": 3,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "random_state": 42
    }
    
    # Log parameters
    mlflow.log_param("model_type", "XGBoost")
    mlflow.log_params(params)
    
    # Train
    model = XGBRegressor(**params, n_jobs=-1, verbosity=0)
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Log metrics
    mlflow.log_metrics({
        "rmse": rmse,
        "mae": mae,
        "r2_score": r2
    })
    
    # Log model (XGBoost native format)
    mlflow.xgboost.log_model(model, "model")
    
    # Save run ID for later
    best_run_id = run.info.run_id
    
    print(f"✅ Logged: RMSE=${rmse:,.2f}, R²={r2:.4f}")

## Step 5: Compare Runs

In [ ]:
# Search all runs in experiment
runs = mlflow.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["metrics.rmse ASC"]
)

print("📊 All Runs (sorted by RMSE):")
display(runs[[
    'run_id', 
    'tags.mlflow.runName',
    'params.model_type',
    'metrics.rmse',
    'metrics.mae',
    'metrics.r2_score'
]].head())

best_run = runs.iloc[0]
print(f"\n🏆 Best Model: {best_run['params.model_type']}")
print(f"   Run ID: {best_run['run_id']}")
print(f"   RMSE: ${best_run['metrics.rmse']:,.2f}")

## Step 6: Register Best Model

Move model to MLflow Model Registry for production use.

In [ ]:
# Register model
MODEL_NAME = "sales_forecasting_model"
model_uri = f"runs:/{best_run['run_id']}/model"

model_version = mlflow.register_model(
    model_uri=model_uri,
    name=MODEL_NAME
)

print(f"✅ Model registered!")
print(f"   Name: {model_version.name}")
print(f"   Version: {model_version.version}")
print(f"   Stage: {model_version.current_stage}")

## Step 7: Transition Model to Staging

In [ ]:
# Create MLflow client
client = mlflow.MlflowClient()

# Transition to Staging
client.transition_model_version_stage(
    name=MODEL_NAME,
    version=model_version.version,
    stage="Staging",
    archive_existing_versions=False
)

print(f"✅ Model transitioned to Staging")
print(f"   Ready for validation tests!")

## Step 8: Load Model from Registry

In [ ]:
# Load model from registry (Staging stage)
model_uri = f"models:/{MODEL_NAME}/Staging"
loaded_model = mlflow.pyfunc.load_model(model_uri)

print(f"✅ Model loaded from registry")
print(f"   Model: {MODEL_NAME}")
print(f"   Stage: Staging")

# Make test prediction
sample_prediction = loaded_model.predict(X_test.head(5))
print(f"\n📊 Sample predictions:")
for i, pred in enumerate(sample_prediction):
    actual = y_test.iloc[i]
    print(f"   {i+1}. Predicted: ${pred:,.2f}, Actual: ${actual:,.2f}, Error: ${abs(pred-actual):,.2f}")

## Step 9: View on DagsHub

In [ ]:
print("🌐 View your experiments on DagsHub:")
print(f"   {MLFLOW_TRACKING_URI.replace('.mlflow', '')}")
print("\n   Navigate to: Experiments tab > {EXPERIMENT_NAME}")
print("   You'll see all 3 runs with metrics and parameters!")

## ✅ Summary

### What We Accomplished:
1. ✅ Connected to MLflow (DagsHub)
2. ✅ Logged 3 model experiments with parameters and metrics
3. ✅ Compared runs systematically
4. ✅ Registered best model to Model Registry
5. ✅ Transitioned model to Staging
6. ✅ Loaded model from registry

### Why This Matters for MLOps:
- 🔄 **Reproducibility**: Every experiment tracked
- 📊 **Comparison**: Easy to see what works
- 🎯 **Model Registry**: Central repository for production models
- 🚀 **Deployment Ready**: Load models by stage (Staging/Production)
- 👥 **Collaboration**: Team can see all experiments

### Model Lifecycle:
```
Training → Register → Staging → Validate → Production → Archive
```

---

**Next:** `05_DVC_Data_Versioning.ipynb` - Version control for data

**© 2024 Amey Talkatkar**